## Downloading Email Headers

In [1]:
username = "krishnaraj.kpt@outlook.com"
password = "BBQtJTSs8uQh57aw"
imap_server = "outlook.office365.com"

In [5]:
import imaplib
import email
from email.header import decode_header
from email.parser import BytesParser
import json

Creating Imap Object

In [6]:
# create an IMAP4 class with SSL
imap = imaplib.IMAP4_SSL(imap_server)
# authenticate
imap.login(username, password)
status, messages = imap.select("INBOX")

# number of top emails to fetch
N = 15

# total number of emails
messages = int(messages[0])
print("The Total Number of Messages in your account are: ", messages)

The Total Number of Messages in your account are:  1017


Downloading Headers

In [7]:
# create a list to store the emails
emails = []

# fetch the top N email headers
for i in range(messages - N, messages + 1):
    # fetch the email header
    result, data = imap.fetch(str(i + 1), "(RFC822.HEADER)")
    if result == "OK":
        # parse the email header
        email_parser = BytesParser()
        email_header = email_parser.parsebytes(data[0][1])

        # create a dictionary to store the email header
        email = {
            "subject": email_header["Subject"],
            "headers": [],
        }

        # add the email headers to the dictionary
        for header in email_header.items():
            email["headers"].append({
                "header": header[0],
                "value": header[1],
            })

        # add the email to the list of emails
        emails.append(email)

# close the IMAP connection
imap.close()

# write the emails to a JSON file
with open("emails.json", "w") as f:
    json.dump(emails, f, indent=4)

Converting the json file to a python dictionary

In [23]:
import json
with open("emails.json", "r") as f:
    emails = json.load(f)

Looking at Header keys of one of the emails

In [29]:
header_names = [i['header'] for i in emails[1]["headers"]]

Focusing on 8 Test Emails

In [24]:
emails = emails[2:10]

Looking at their Subjects

In [26]:
[i['subject'] for i in emails]

['Sending from phone gmail app',
 'Self mail from web outlook client',
 'Self test mail from gmail web client',
 'self test mail from mit gmail client',
 'From movile outlook app',
 'Mail sent from phone',
 "Mother's phone Gmail app",
 'Sending from browser with vpn connection on']

Analyse each header and find out what it means

In [59]:
print("Total number of headers in provided email: ", len(header_names))

Total number of headers in provided email:  77


In [45]:
# print header names with their index
for index, header in enumerate(header_names):
    print(index, header)

0 MIME-Version
1 Received
2 ARC-Seal
3 ARC-Message-Signature
4 ARC-Authentication-Results
5 Received
6 Received
7 Authentication-Results
8 Received-SPF
9 Received
10 X-IncomingTopHeaderMarker
11 ARC-Seal
12 ARC-Message-Signature
13 ARC-Authentication-Results
14 DKIM-Signature
15 Received
16 Received
17 From
18 To
19 Subject
20 Thread-Topic
21 Thread-Index
22 Date
23 Message-ID
24 Accept-Language
25 Content-Language
26 X-MS-Has-Attach
27 X-MS-TNEF-Correlator
28 msip_labels
29 x-tmn
30 x-ms-traffictypediagnostic
31 X-MS-Office365-Filtering-Correlation-Id
32 X-Microsoft-Antispam-Untrusted
33 X-Microsoft-Antispam-Message-Info-Original
34 X-MS-Exchange-AntiSpam-MessageData-Original-ChunkCount
35 X-MS-Exchange-AntiSpam-MessageData-Original-0
36 Content-Type
37 X-MS-Exchange-Transport-CrossTenantHeadersStamped
38 X-IncomingHeaderCount
39 Return-Path
40 X-MS-Exchange-Organization-ExpirationStartTime
41 X-MS-Exchange-Organization-ExpirationStartTimeReason
42 X-MS-Exchange-Organization-Expiratio

### MIME-Version

MIME-Version indicates the email's message format. It's important in investigations to understand how the message is structured and if it includes multimedia or attachments.

In [38]:
emails[1]['headers'][0]['header'], emails[1]['headers'][0]['value']

('MIME-Version', '1.0')

### Received

Received headers track the path of the email through various servers. This is crucial for tracing the email's journey, identifying potential anomalies, or investigating its source.


In [40]:
emails[1]['headers'][1]['header'], emails[1]['headers'][1]['value']

Received from SJ0PR17MB4837.namprd17.prod.outlook.com (2603:10b6:a03:37a::10)
 by DS7PR17MB6730.namprd17.prod.outlook.com with HTTPS; Sun, 29 Oct 2023
 17:32:29 +0000


### ARC-Seal

ARC (Authenticated Received Chain) headers help verify the authenticity of email forwarding. ARC-Seal ensures the integrity of email headers, reducing the risk of spoofing.


In [41]:
emails[1]['headers'][2]['header'], emails[1]['headers'][2]['value']

('ARC-Seal',
 'i=2; a=rsa-sha256; s=arcselector9901; d=microsoft.com; cv=pass;\r\n b=UERZv4BlU4weLVUXdoohvbjGYpYf4pb9pFtOwayMK+mtwTbjMhWrqskYHiEhqgH1rrxhnvYgrK7YkvSkKXiEypdAOak0f4KcLaHNb/KEBpCVvQoKhVUX2zWzFMxVLsIRkMgoltrKRs0JShcFwrbt6XCvxCZUTbsGQs/hFpaN0sYlFys1Qu41etiVDrmS8ZYpq4ZnHuXxdBzxW6A8Aql06f5sr4CF2fSeAIjAFu5JB5/tTHlu9wFIYa49rmvL4i2S8QLHI8IdHsvPpz0oNrK0BVzf8bFat2iF7qnIX2J1lkXo21nWrGEioqJjPP6uusrYJBc5R+SatB5i3rmSYpqn0w==')

### ARC-Message-Signature

ARC-Message-Signature is part of ARC headers and provides cryptographic assurance of email header integrity, aiding in detecting email tampering.


In [42]:
emails[1]['headers'][3]['header'], emails[1]['headers'][3]['value']

('ARC-Message-Signature',
 'i=2; a=rsa-sha256; c=relaxed/relaxed; d=microsoft.com;\r\n s=arcselector9901;\r\n h=From:Date:Subject:Message-ID:Content-Type:MIME-Version:X-MS-Exchange-AntiSpam-MessageData-ChunkCount:X-MS-Exchange-AntiSpam-MessageData-0:X-MS-Exchange-AntiSpam-MessageData-1;\r\n bh=6+cClN22oIrc2YH61nPV02IScRJCb64tf03+C6e0M4c=;\r\n b=eM0/ekjq2KehxMnRBghHFpHhgVKFTyGOnc/ccplTJ7KonRY/xFz58qfR9ixNv6igINRBz+QQaWFKprBg57YvrEJRPljNUKG0WmKJXKF1Czif79KmQpGcyxrjMBkNfga0hZWdPpPgOAVbNEG0z2uYUQ4zIqpiqq0wJ69EUaestV84DGs3O14jqflXjihcktRdZBX6zL/WD1gnOdr6ParkWfeJUaCV1BjMcpoUFXMnCZwlB2ST1aNgUdXHrqvxty4c2Q0/1uDWWr9Wu/Vx5h/kN9LqqjvwNmvzTLuRE1oPLaIch8T2dBnIf6leWeKENffAjFK+Kba43a68mbAnbxlvsg==')

### ARC-Authentication-Results

These headers indicate the email's authentication status. Investigators can use this to assess the email's legitimacy and potential for phishing.


In [43]:
emails[1]['headers'][4]['header'], emails[1]['headers'][4]['value']

('ARC-Authentication-Results',
 'i=2; mx.microsoft.com 1; spf=pass (sender ip is\r\n 40.92.20.10) smtp.rcpttodomain=outlook.com smtp.mailfrom=outlook.com;\r\n dmarc=pass (p=none sp=quarantine pct=100) action=none\r\n header.from=outlook.com; dkim=pass (signature was verified)\r\n header.d=outlook.com; arc=pass (0 oda=0 ltdi=1)')

### DKIM-Signature

DKIM (DomainKeys Identified Mail) verifies that the email content hasn't been altered in transit. It's crucial for email integrity checks and source verification.


In [46]:
emails[1]['headers'][14]['header'], emails[1]['headers'][14]['value']

('DKIM-Signature',
 'v=1; a=rsa-sha256; c=relaxed/relaxed; d=outlook.com;\r\n s=selector1;\r\n h=From:Date:Subject:Message-ID:Content-Type:MIME-Version:X-MS-Exchange-SenderADCheck;\r\n bh=6+cClN22oIrc2YH61nPV02IScRJCb64tf03+C6e0M4c=;\r\n b=aZNfYlN+/lbQqIqDR1Cblhb9/x28HgNh+pAywoD+43Be+F/5cGPhWuKGP6InowbprEEutN/A5RLJ20qRurKixCRrzUMNtv/QnJXQFkFCvHBOWiZyEANGeA5iUHQ38WwdNG0IsLWgyb4s82CjKshEyYgkgfPcpP854CN9IsqJ1EUA3TIayHNbGwRilfGqwSFICBp4EODwvlXl+WWU4ihYg7HwlSaKgC/gn2WSnmIo3G/L3YbZkI9X41B3ttxHTCeCTjAktqE29Ww13fLEgMB4F+gD8xJSmNQlYvSAcM6unCowj5AjXRLPHJ5aCseQL0Mnq5aFVufoGrR9yFUn45RoYA==')

### From

The "From" header shows the sender's email address. It's essential for identifying the sender, although it can be spoofed.


In [47]:
emails[1]['headers'][17]['header'], emails[1]['headers'][17]['value']

('From', 'Krishnaraj Thadesar <Krishnaraj.kpt@outlook.com>')

### To

The "To" header reveals the email's recipient, which is significant for understanding the email's target and potential threat actors.


In [48]:
emails[1]['headers'][18]['header'], emails[1]['headers'][18]['value']

('To', 'Krishnaraj Thadesar <krishnaraj.kpt@outlook.com>')

### Subject

The subject line provides insight into the email's content, which is crucial for assessing the email's purpose and relevance to an investigation.


In [49]:
emails[1]['headers'][19]['header'], emails[1]['headers'][19]['value']

('Subject', 'Self mail from web outlook client')

### Date

The date header shows when the email was sent. It's valuable for establishing timelines and correlations with other events.


In [50]:
emails[1]['headers'][22]['header'], emails[1]['headers'][22]['value']

('Date', 'Sun, 29 Oct 2023 17:32:25 +0000')

### Message-ID

The Message-ID is unique to each email and can be used for tracking and associating related messages in an investigation.


In [51]:
emails[1]['headers'][23]['header'], emails[1]['headers'][23]['value']

('Message-ID',
 '\r\n <DS7PR17MB6730EE7BE9EA14814BB3AFE980A2A@DS7PR17MB6730.namprd17.prod.outlook.com>')

### Content-Type

Content-Type specifies the format of the email content. It helps investigators interpret the email's structure and potential for malicious attachments.


In [52]:
emails[1]['headers'][36]['header'], emails[1]['headers'][36]['value']

('Content-Type',
 'multipart/alternative;\r\n\tboundary="_000_DS7PR17MB6730EE7BE9EA14814BB3AFE980A2ADS7PR17MB6730namp_"')

### Return-Path

Return-Path indicates where undeliverable emails should be sent. It can assist in identifying email redirection or bouncing patterns.


In [54]:
emails[1]['headers'][39]['header'], emails[1]['headers'][39]['value']

('Return-Path', 'krishnaraj.kpt@outlook.com')

### X-Sender-IP

This header contains the IP address of the sender, which is essential for tracking the origin of the email and potential geolocation.


In [55]:
emails[1]['headers'][56]['header'], emails[1]['headers'][56]['value']

('X-Sender-IP', '40.92.20.10')

### X-MS-Exchange-Transport-EndToEndLatency

End-to-end latency is crucial for assessing the email's delivery speed, which might reveal anomalies or delays in transit.


In [56]:
emails[1]['headers'][71]['header'], emails[1]['headers'][71]['value']

('X-MS-Exchange-Transport-EndToEndLatency', '00:00:02.4306131')

### X-Microsoft-Antispam-Mailbox-Delivery

This header provides information about the email's delivery and its classification as spam or not, aiding in filtering and threat analysis.


In [57]:
emails[1]['headers'][73]['header'], emails[1]['headers'][73]['value']

('X-Microsoft-Antispam-Mailbox-Delivery',
 '\r\n\tucf:0;jmr:0;ex:0;auth:1;dest:I;OFR:SpamFilterPass;ENG:(5062000305)(920221119095)(90000117)(920221120095)(90013020)(91025020)(91040095)(9050020)(9065024)(9100341)(1000006)(944500132)(2008001134)(4810010)(4910033)(9920006)(9510006)(10105021)(9320005)(9230038)(120001);')

### X-Message-Info and X-Message-Delivery

These headers contain miscellaneous information about the email's handling, which can be valuable for tracking and understanding the email's journey.

In [58]:
emails[1]['headers'][75]['header'], emails[1]['headers'][75]['value']

('X-Message-Delivery', 'Vj0xLjE7dXM9MDtsPTA7YT0xO0Q9MTtHRD0xO1NDTD0tMQ==')

Defining a function to get information about an ip address for sender ip analysis

In [65]:
import requests

def get_ip_information(ip_address):
    def get_ip_location():
        # Make a GET request to ipinfo.io with the IP address
        url = f"https://ipinfo.io/{ip_address}/json"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            return data
        else:
            return None

    location_info = get_ip_location()
    # print(location_info)
    if location_info:
        # Print the location information
        print(f"IP Address: {location_info['ip']}")
        print(f"Hostname: {location_info['hostname']}")
        print(f"City: {location_info['city']}")
        # print(f"Region: {location_info['region']}")
        print(f"Country: {location_info['country']}")
        print(f"Location: {location_info['loc']}")
        # print(f"Organization: {location_info['org']}")
        print(f"Timezone: {location_info['timezone']}")
    else:
        print("Unable to retrieve location information for the IP address.")


Analysing Sender IP Address for all emails

In [66]:
for email in emails:
    for header in email['headers']:
        if header['header'] == 'X-Sender-IP':
            print("Subject of Email: ", email['subject'])
            print("IP Address Information:")
            get_ip_information(header['value'])

            print()

Subject of Email:  Sending from phone gmail app
IP Address Information:
IP Address: 209.85.218.48
Hostname: mail-ej1-f48.google.com
City: Oudeschip
Country: NL
Location: 53.4300,6.8264
Timezone: Europe/Amsterdam

Subject of Email:  Self mail from web outlook client
IP Address Information:
IP Address: 40.92.20.10
Hostname: mail-bn8nam11olkn2010.outbound.protection.outlook.com
City: Boydton
Country: US
Location: 36.6676,-78.3875
Timezone: America/New_York

Subject of Email:  Self test mail from gmail web client
IP Address Information:
IP Address: 209.85.167.178
Hostname: mail-oi1-f178.google.com
City: Tulsa
Country: US
Location: 36.1540,-95.9928
Timezone: America/Chicago

Subject of Email:  self test mail from mit gmail client
IP Address Information:
IP Address: 209.85.219.47
Hostname: mail-qv1-f47.google.com
City: Raleigh
Country: US
Location: 35.7721,-78.6386
Timezone: America/New_York

Subject of Email:  From movile outlook app
IP Address Information:
IP Address: 40.92.242.29
Hostname